# Analyze results
Aug 28, 2020

In [91]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [92]:
from ipywidgets import *

In [93]:
%matplotlib widget

In [94]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [95]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/')
import post_analysis_pandas as post


In [96]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [97]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [98]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [99]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201212_190922_cgan_long_run_model5', '20201210_073…

In [100]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201213_064819_cgan_long_run_model3


## Plot Losses

In [101]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [102]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
93741,93741.0,29.0,0.865936,0.195456,1.061392,2.656963,35.026142,32.369179,-9.800303,32.165363,-10.421804,0.802174,-4.506536,-2.437708,0.160003
93742,93742.0,29.0,0.142094,0.383329,0.525423,3.930223,36.010735,32.080513,-10.152968,32.172020,-10.404503,3.946457,-1.764868,-3.865921,0.158547
93743,93743.0,29.0,0.297089,0.146250,0.443339,3.802860,35.600521,31.797661,-9.790960,32.119316,-9.691799,1.811565,-4.020899,-3.681612,0.163039
93744,93744.0,29.0,0.276689,0.251084,0.527773,2.608525,34.606506,31.997980,-10.533615,32.139465,-10.605375,2.501570,-3.451478,-2.362454,0.160141
93745,93745.0,29.0,0.201063,0.632170,0.833233,5.948665,38.245205,32.296539,-9.948372,32.131905,-10.942076,4.403502,-1.270124,-5.867620,0.161978
93746,93746.0,29.0,1.040504,0.207405,1.247909,1.909460,33.160252,31.250792,-10.372437,32.198689,-10.453860,0.164225,-5.431497,-1.171568,0.161135
93747,93747.0,29.0,0.202877,1.259632,1.462508,7.605000,40.081516,32.476517,-10.675793,32.255119,-10.455141,3.948994,0.239651,-7.510494,0.162908
93748,93748.0,29.0,1.115848,0.295704,1.411553,4.621473,37.112183,32.490707,-9.695112,32.320522,-9.913340,0.379364,-8.843170,-4.511860,0.159162
93749,93749.0,29.0,0.197222,0.432103,0.629325,3.107614,35.245949,32.138336,-9.225245,32.020081,-9.842088,4.116516,-2.325570,-2.888541,0.159890
93750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.844509,-9.986048,NaN,NaN,NaN,NaN


In [103]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [104]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-10.765849409103392


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
90781,90781.0,29.0,0.152281,0.756760,0.909041,8.288021,39.721001,31.432981,-10.857597,32.015594,-11.504271,4.814192,-1.368094,-8.279819,0.159601
91381,91381.0,29.0,0.685821,0.119513,0.805334,2.057580,34.583618,32.526039,-10.990952,32.488823,-11.476854,1.041671,-5.324142,-1.453889,0.163690
92186,92186.0,29.0,0.147404,1.676678,1.824082,7.335183,39.122944,31.787760,-10.696949,31.486151,-11.456476,5.526217,0.610389,-7.325423,0.161574
91380,91380.0,29.0,0.186722,0.649336,0.836057,5.150787,37.299225,32.148438,-11.385084,32.411255,-11.416026,3.373841,-1.024374,-5.093239,0.163756
92938,92938.0,29.0,0.221063,0.130110,0.351173,4.218696,36.128815,31.910118,-10.711822,31.888868,-11.414535,2.896077,-4.467079,-4.140048,0.157302
92717,92717.0,29.0,0.217225,0.218246,0.435471,4.033490,35.256096,31.222605,-10.368874,31.670742,-11.394478,3.088308,-3.229792,-3.853784,0.157919
92786,92786.0,29.0,0.369608,0.187653,0.557261,2.752243,34.880760,32.128517,-10.260275,32.012634,-11.387912,1.628804,-3.253624,-2.501306,0.159600
92918,92918.0,29.0,2.197199,0.321671,2.518870,3.471523,35.736977,32.265453,-10.603239,31.941231,-11.378947,-1.528638,-8.887394,-3.239290,0.159159
91370,91370.0,29.0,0.122229,0.704094,0.826323,5.418262,37.099239,31.680977,-10.608492,32.856133,-11.374300,3.931532,-0.847478,-5.384661,0.162140
90779,90779.0,29.0,0.835112,0.070045,0.905157,2.852937,35.086132,32.233196,-10.790414,31.875519,-11.366351,0.766425,-5.293872,-2.532626,0.161475


In [105]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
90781,90781.0,29.0,0.152281,0.756760,0.909041,8.288021,39.721001,31.432981,-10.857597,32.015594,-11.504271,4.814192,-1.368094,-8.279819,0.159601
91381,91381.0,29.0,0.685821,0.119513,0.805334,2.057580,34.583618,32.526039,-10.990952,32.488823,-11.476854,1.041671,-5.324142,-1.453889,0.163690
92186,92186.0,29.0,0.147404,1.676678,1.824082,7.335183,39.122944,31.787760,-10.696949,31.486151,-11.456476,5.526217,0.610389,-7.325423,0.161574
91380,91380.0,29.0,0.186722,0.649336,0.836057,5.150787,37.299225,32.148438,-11.385084,32.411255,-11.416026,3.373841,-1.024374,-5.093239,0.163756
92938,92938.0,29.0,0.221063,0.130110,0.351173,4.218696,36.128815,31.910118,-10.711822,31.888868,-11.414535,2.896077,-4.467079,-4.140048,0.157302
92717,92717.0,29.0,0.217225,0.218246,0.435471,4.033490,35.256096,31.222605,-10.368874,31.670742,-11.394478,3.088308,-3.229792,-3.853784,0.157919
92786,92786.0,29.0,0.369608,0.187653,0.557261,2.752243,34.880760,32.128517,-10.260275,32.012634,-11.387912,1.628804,-3.253624,-2.501306,0.159600
92918,92918.0,29.0,2.197199,0.321671,2.518870,3.471523,35.736977,32.265453,-10.603239,31.941231,-11.378947,-1.528638,-8.887394,-3.239290,0.159159


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
60831,60831.0,19.0,0.068140,2.274131,2.342271,9.372945,41.692017,32.319073,-8.678756,30.761848,-8.609508,8.712275,1.246752,-9.362792,0.159836
41364,41364.0,13.0,0.180779,0.672025,0.852803,4.393993,36.904259,32.510265,-8.200768,30.789913,-8.719219,3.268727,-0.748029,-4.348578,0.159725
43241,43241.0,13.0,0.123524,2.814135,2.937659,7.077887,38.963844,31.885958,-9.138475,30.793978,-9.545019,6.560585,2.326764,-7.058206,0.152784
50362,50362.0,16.0,0.265461,0.141984,0.407445,3.707460,35.682018,31.974560,-9.051419,30.812077,-9.390673,3.035304,-2.961789,-3.640084,0.151666
36038,36038.0,11.0,0.409049,1.355906,1.764956,9.812836,42.253120,32.440285,-8.035342,30.822903,-7.701461,6.075340,0.062513,-9.772310,0.155080
60833,60833.0,19.0,1.029063,0.330346,1.359409,0.641296,33.039654,32.398357,-8.676680,30.822926,-8.857542,0.329012,-3.032990,1.871139,0.158293
38154,38154.0,12.0,0.140291,1.395347,1.535637,5.228235,37.560371,32.332134,-9.386969,30.881203,-9.113935,4.311404,0.666425,-5.200540,0.152809
43646,43646.0,13.0,0.181336,2.625080,2.806416,6.657950,39.315407,32.657455,-9.345995,30.888105,-9.397270,5.049576,1.624019,-6.605533,0.151069


## Read stored chi-squares for images

In [ ]:

df_list=[]
labels_lst=[0,1,2,3]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(label)))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list

In [ ]:

def f_plot_hist_spec(df,class_labels,steps_list,bkg_dict,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    for class_label in class_labels:
        df=df[df.step.isin(steps_list)]
        print(df.shape)
        suffix='_%s'%(class_label)
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(class_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,:,:]
                f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
        ### Plot reference data
        dict_bkg=bkg_dict[str(class_label)]
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
    
    if plot_type in ['hist','spec']:     
        plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
        plt.yscale('log')

# f_plot_hist_spec(df_merged,labels_lst,best_step,bkgnd_dict,'hist')

### Read validation data

In [ ]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_list=[0.5,0.65,0.8,1.1]
bkgnd_dict={}
num_bkgnd=2000

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_list[label])
    samples=np.load(fname,mmap_mode='r+')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(label)]=dict_val
    del samples

In [81]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',labels_lst=[0,1,2,3],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1c']:
            for i in labels_lst: chi_cols.append(j+'_'+str(i))
    else: ## Add chi-square for specific label
        assert label in labels_lst, "label %s is not in %s"%(label,labels_lst)
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append(j+'_'+str(label))
    
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

In [82]:
# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=-1,use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [83]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),label=ToggleButtons(options=['all']+labels_lst),
labels_lst=fixed(labels_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [85]:
df_sliced=w.result
# df_sliced

In [86]:
best_step=[]
best_step.append(f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None]).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_spec1',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None]).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1c',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None]).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[91590]


In [53]:
# best_step=[6176]
# best_step=np.arange(40130,40135).astype(int)

In [87]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(1, 79)
[(29, 91590)]


In [88]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]

df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1c
9158,29-91590,4.364373,177.755915,1.489715


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [90]:
interact_manual(f_plot_hist_spec,df=fixed(df_merged),
                class_labels=SelectMultiple(options=labels_lst),
                steps_list=SelectMultiple(options=best_step),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='class_labels', options=(0, 1, 2, 3), value=()), SelectMultip…

<function __main__.f_plot_hist_spec(df, class_labels, steps_list, bkg_dict, plot_type)>